### Unique Identifier Extraction

(currently notes)

Anecdata - folks don't really comply with the URN spec but will include URN-like things.

I don't have any plan for identifying things tagged as "cool" URIs but aren't full URLs. 


####URNs We Can't Identify

Using some regex that is not really functional:

```
^([a-z0-9.#]{0,}:([a-z0-9][a-z0-9.-]{0,31}):[a-z0-9A-Z_()+,-.:=@;$!*\'%/?#\[\]]+)
```

and that is expecting 3 parts.

Not a URN (not saying they should be, either):

 - nlm:nlmuid-67840340R-bk
 - geonode:combined_tasks
 - gov.noaa.ngdc.stp.indices:G00587
 - gov.noaa.nmfs.inport:18623

 
 
Things that are a URN according to the regex:

 - urn:nbn:nl:ui:13-01hb-3b (extracted from a rule set)
 - oai:easy.dans.knaw.nl:easy-dataset:53685 (coded as a url)
 - oai:www.mpi.nl:MPI110411
 - clarin.eu:cr1:p_1271859438204
 - urn:uuid:6426c02b-f2b1-4326-a767-2384c303faf3
 - oai:oai.datacite.org:1979342
 
 
 
 

####More generally things that are awkward

(We forgot whitespace in some string join effort):

St1958-03-31T00:00:00-111.155555685007http://services.azgs.az.gov/OnlineAccessMineFiles/M-R/MilewidePima15.pdfDocument